In [18]:
import pandas
import numpy as np
from colour import Color

has_attention = True
dont_show_ok = True
data = pandas.read_csv("test0.best.csv")
# print(data.head)
data["attention"] = data["attention"].apply(lambda x: np.array(eval(x)))
data["ok"] = data["prediction"] == data["label"]


In [19]:

data

,sentence,label,prediction,ok,bert_projection,attention,doc-vectors,score-negative,score-positive,token_subword,urn,fp,score-prediction,probs
0,Videns eos Athenagora utrosque in amplexu cum ...,negative,negative,True,NaN,"[0.006909254472702742, 0.006201545242220163, 0...","[0.27551907300949097, -0.4314749240875244, -0....",0.928650,0.071350,[CLS] videns_ eos_ athen agor a_ utro sque_ in...,urn:cts:latinLit:stoa0323.stoa001.digilibLT-lat1,./dataset/negative-examples/latinLit_stoa0323....,0.928650,[0.9286502 0.07134982]
1,post hunc autem ascendit Nabuchodonosor rex Ba...,negative,negative,True,NaN,"[0.01796604134142399, 0.014929000288248062, 0....","[0.7306641340255737, -0.17974154651165009, -0....",0.998411,0.001589,[CLS] post_ hunc_ autem_ ascendit_ nabuchodono...,urn:cts:greekLit:tlg0527.tlg017.obi-lat1,./dataset/negative-examples/greekLit_tlg0527.t...,0.998411,[0.9984112 0.00158881]
2,"Ipse in primis absque peccato hominem fecit , ...",negative,negative,True,NaN,"[0.01138648483902216, 0.01280389353632927, 0.0...","[0.6239117980003357, -0.0869000107049942, -0.7...",0.996760,0.003240,[CLS] ipse_ in_ primis_ absque_ peccato_ homin...,urn:cts:latinLit:stoa0121g.stoa005.opp-lat1,./dataset/negative-examples/latinLit_stoa0121g...,0.996760,[0.9967601 0.00323992]
3,munus ecce fictile inimus intra regiam salutis .,negative,negative,True,NaN,"[0.04482552036643028, 0.026790864765644073, 0....","[0.08615805208683014, -0.24661311507225037, -0...",0.990516,0.009484,[CLS] munus_ ecce_ ficti le_ inim us_ intra_ r...,urn:cts:latinLit:stoa0238.stoa009.perseus-lat2,./dataset/negative-examples/latinLit_stoa0238....,0.990516,[0.990516 0.00948396]
4,Non pol mihi quidem ;,negative,negative,True,NaN,"[0.13044895231723785, 0.06353741139173508, 0.2...","[0.19873300194740295, 0.023621028289198875, -0...",0.903034,0.096966,[CLS] non_ pol_ mihi_ quidem_ [SEP],urn:cts:latinLit:phi0119.phi020.perseus-lat2,./dataset/negative-examples/latinLit_phi0119.p...,0.903034,[0.9030338 0.09696619]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,certe ex utero sanctitatem custodisse et infan...,negative,negative,True,NaN,"[0.00540215615183115, 0.011581731960177422, 0....","[0.23966342210769653, -0.1174972802400589, -0....",0.989645,0.010355,[CLS] certe_ ex_ utero_ sanctitatem_ custodi s...,urn:cts:latinLit:stoa0104p.stoa005.opp-lat1,./dataset/negative-examples/latinLit_stoa0104p...,0.989645,[0.9896449 0.01035509]
2738,"Cum mora non tuta est , totis incumbere remis ...",positive,negative,False,NaN,"[0.01787783019244671, 0.030204765498638153, 0....","[-0.049520790576934814, -0.3514571189880371, -...",0.981141,0.018859,"[CLS] cum_ mora_ non_ tuta_ est_ ,_ totis_ inc...",urn:cts:latinLit:phi0959.phi004.perseus-lat2,/home/thibault/dev/these-corpus/data/1386.xml,0.981141,[0.981141 0.01885901]
2739,"nihil requirit nisi dei gratiam , solum illud ...",negative,negative,True,NaN,"[0.02220521867275238, 0.02586475946009159, 0.0...","[0.6747543811798096, -0.006729818414896727, -0...",0.999557,0.000443,"[CLS] nihil_ requirit_ nisi_ dei_ gratiam_ ,_ ...",urn:cts:latinLit:stoa0022.stoa034.opp-lat1,./dataset/negative-examples/latinLit_stoa0022....,0.999557,[9.995567e-01 4.432459e-04]
2740,non arbitror te velle similem esse Epicureorum...,negative,negative,True,NaN,"[0.005178079474717379, 0.003804730949923396, 0...","[0.33246707916259766, -0.19699843227863312, -0...",0.977771,0.022229,[CLS] non_ arbitror_ te_ velle_ similem_ esse_...,urn:cts:latinLit:phi0474.phi050.perseus-lat1,./dataset/negative-examples/latinLit_phi0474.p...,0.977771,[0.9777706 0.02222938]


In [21]:
from IPython.core.display import display as _display, HTML  as _HTML
data.iloc[0]

COLORS = [Color("white")] + list(Color("yellow").range_to(Color("red"),9))

def visualize(sentence, attention, tag="p", colors=None, display=True, label=None, is_right: bool = False,
             show_score: bool = False, use_subword=False):
    if not colors:
        colors = COLORS

    # print(attention)
    
    if use_subword:
        sentence = sentence["token_subword"]
    else:
        sentence = sentence["sentence"]
        
    html = f"<{tag}>"
    if label:
        html += f"<b style=\"{'text-decoration: line-through;' if not is_right else ''};\">{label}</b> -> "
    if show_score:
        html += f"<i>{max(attention):.2f}</i> -> "
    
    if attention is not None:
        spans = [
            f"<span style=\"background-color: {colors[max(0, round(s*10) - 1)]}; display: inline-block;\">{w}</span>"
            for w, s in zip(sentence.split(" "), attention)
        ]
    else:
        spans = [
            f"<span style=\"display: inline-block;\">{w}</span>"
            for w in sentence.split(" ")
        ]
        
    html += " ".join(spans) + f"</{tag}>"

    if display:
        _display(_HTML(html))
    return html

html = "<div>"
for idx, row in data.iterrows():
    if dont_show_ok and row["ok"] == True:
        continue
    html += "    %s\n" % visualize(
        row,
        row.get("attention") if has_attention else None,
        label=row["label"],
        is_right=row["ok"],
        use_subword=True
    )
html += "</div>"
#print(html)